In [1]:
import torch

import matplotlib.pyplot as plt
import torchmetrics as tm

from config import *
from stages import *
from train import *
from data.ssa import SSA

from data.util import crop_q_between, split_weekdays_and_weekends

%reload_ext autoreload
%autoreload 2

In [2]:
mse = tm.MeanSquaredError().to(CONFIG.device)
mape = tm.MeanAbsolutePercentageError().to(CONFIG.device)
mat_q = CONFIG.load('mat_q.pt')
CONFIG.alpha = 0.2
CONFIG.alpha = 1000
mat_c_all, mat_x_repr, nonempty, representatives = compress_data(
    mat_q.abs(), CONFIG.read_period, CONFIG.train_period, CONFIG.alpha)
mat_q = mat_q[:, mat_q.sum(dim=0) > 0]
normalizing_constants_all = mat_q.max(dim=0)[0]
mat_q /= normalizing_constants_all
mat_q

tensor([[0.0000, 0.0000, 0.0025,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0073, 0.0100,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0150,  ..., 0.0039, 0.0000, 0.0000],
        ...,
        [0.0095, 0.0290, 0.0698,  ..., 0.0000, 0.0463, 0.0000],
        [0.0000, 0.0163, 0.0175,  ..., 0.0000, 0.0256, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0')

In [3]:
mat_q.shape

torch.Size([35040, 357])

##### Compare MSE and MAPE losses for different parameter configurations

Pre-train DBN and Train DBN attaching KELM on each epoch

In [4]:
results_wd = {}
results_we = {}

ssa = SSA(4, [2, 2], CONFIG.device)

CONFIG.spectral_threshold = 1440
CONFIG.dbn_hidden_layer_sizes = [80, 80, 80]

gamma = 1
reg_coeff = 1


for gamma in [gamma]:#range(1, 26, 5):
  for reg_coeff in [reg_coeff]:#range(1, 102, 25):
    for P in [40,]:#range(125, 3125, 125):
      results_P = {}
      CONFIG.spectral_threshold = P
      for N in [84,]:#range(4, 324, 80):
        CONFIG.dbn_hidden_layer_sizes = [N, N, N]

        mat_c_all, mat_x_all, nonempty, representatives = compress_data(
            mat_q.abs(), CONFIG.read_period, CONFIG.train_period, CONFIG.alpha)
        mat_q_trend_all, mat_q_resid_all = preprocess_data(
            CONFIG.spectral_threshold, mat_c_all)
        mat_q_resid_all = crop_q_between(mat_q_resid_all, CONFIG.read_period,
                                         CONFIG.train_period).real
        mat_c_all = crop_q_between(
            mat_c_all, CONFIG.read_period, CONFIG.train_period)

        print('Representatives: ' + str(representatives.tolist()))
        # while mat_c.shape[1] == 1:
        #   CONFIG.alpha += 0.01
        #   print(f'alpha={CONFIG.alpha}')
        #   mat_c, mat_x, nonempty, representatives = compress_data(
        #     mat_q_resid.abs(), CONFIG.read_period, CONFIG.train_period,
        #     CONFIG.alpha)
        # mat_q_trend_all = mat_q_trend[:, nonempty][:, representatives].abs()
        for section_index, section_number in tqdm(enumerate(representatives.tolist())):
          mat_q_trend = mat_q_trend_all[:, section_index][:, None].real
          mat_q_resid = mat_q_resid_all[:, section_index][:, None].real

          norm_const = normalizing_constants_all[section_number].item()

          (_, val_trend_wd_dataloader, _), (_, val_trend_we_dataloader, _) = \
            crop_and_split_mat(mat_q_trend, CONFIG, separate_weekends=False)
          # (_, val_resid_wd_dataloader, _), (_, val_resid_we_dataloader, _) =
          #   crop_and_split_mat(mat_q_resid, CONFIG)
          mat_c_wd_datasets, mat_c_we_datasets = split_mat(mat_q_resid, CONFIG)
          # print(gamma, reg_coeff, P, N, mat_c.shape)

          del mat_q_resid

          dbn, kelm, val_resid_we_dataloader = train_with_config(
              CONFIG, mat_c_we_datasets, dbn_training_epochs=100, stride=1,
              gamma=gamma, reg_coeff=reg_coeff)
          prefix = f'alpha{CONFIG.alpha}_p{P}_n{N}_sec{section_number}_'\
            .replace('.', '_')
          CONFIG.save(dbn.state_dict(), prefix + 'dbn.pt')
          CONFIG.save(kelm.state_dict(), prefix + 'kelm.pt')

          # continue

          mse_loss_trend = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_resid = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_overall = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_trend_unnorm = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_resid_unnorm = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_overall_unnorm = torch.tensor([0.,]).to(CONFIG.device)

          trend_y = torch.tensor([0.,]).to(CONFIG.device)
          resid_y = torch.tensor([0.,]).to(CONFIG.device)
          overall_y = torch.tensor([0.,]).to(CONFIG.device)

          iter_trend = iter(val_trend_we_dataloader)
          n_samples = 0

          resid = {'y': [], 'pred': []}
          trend = {'y': [], 'pred': []}
          overall = {'y': [], 'pred': []}
          for X_resid, y_resid in val_resid_we_dataloader:
              resid_y += y_resid.item()
              X_trend, y_trend = next(iter_trend)
              trend_y += y_trend.item()
              pred_trend = ssa.forecast(
                  X_trend.squeeze(0).T, 1).sum(0)[-1][None]
              mse_loss_trend += mse(pred_trend, y_trend)
              mse_loss_trend_unnorm += mse(pred_trend * norm_const, y_trend * norm_const)
              trend['y'].append(y_trend.item() * norm_const)
              trend['pred'].append(pred_trend.item() * norm_const)
              # print('TREND', y_trend, pred_trend)

              pred_resid = dbn(X_resid).squeeze(0)
              pred_resid = kelm(pred_resid).T
              mse_loss_resid += mse(pred_resid, y_resid)
              mse_loss_resid_unnorm += mse(pred_resid * norm_const, y_resid * norm_const)
              resid['y'].append(y_resid.item() * norm_const)
              resid['pred'].append(pred_resid.item() * norm_const)

              # print('RESID', y_resid, pred_resid)

              pred = pred_trend + pred_resid

              y = y_trend + y_resid
              overall_y += y.item()
              overall['y'].append(y.item() * norm_const)
              overall['pred'].append(pred.item() * norm_const)
              # print('OVRLL', y, pred)
              # print('=====')

              mse_loss_overall += mse(pred, y).item()
              mse_loss_overall_unnorm += mse(pred * norm_const, y * norm_const).item()
              n_samples += 1

          mse_loss_trend /= n_samples
          mse_loss_resid /= n_samples
          mse_loss_overall /= n_samples
          mse_loss_resid_unnorm /= n_samples
          mse_loss_trend_unnorm /= n_samples
          mse_loss_overall_unnorm /= n_samples

          resid_y /= n_samples
          trend_y /= n_samples
          overall_y /= n_samples

          results_P[section_number] = {
              'resid': resid,
              'trend': trend,
              'overall': overall
          }

          # print(f'gamma={gamma}, reg_coeff={reg_coeff}, ' +
          #       f'WE P={CONFIG.spectral_threshold}, ' +
          #       f'N={CONFIG.dbn_hidden_layer_sizes}, ' +
          #       f'trend={trend_y.item()}+-{mse_loss_trend.item()}' +
          #       f'~{mse_loss_trend_unnorm.item()}, ' +
          #       f'resid={resid_y.item()}+-{mse_loss_resid.item()}' +
          #       f'~{mse_loss_resid_unnorm.item()}, ' +
          #       f'overall={overall_y.item()}+-{mse_loss_overall.item()}' +
          #       f'~{mse_loss_overall_unnorm.item()}')
          del dbn
          del kelm
      results_we[P] = results_P

Representatives: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 21

0it [00:00, ?it/s]/home/kolusask/TUM/IDP/IDP/src/data/ssa.py:153: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3614.)
  P = U.T[:, :-1].permute(2, 0, 1)
356it [38:50:08, 392.72s/it]


In [5]:
import json
with open('results_we_norm_all.json', 'w') as file:
  json.dump(results_we, file)

##### Use compression

In [6]:
# results_wd = {}
# results_we = {}

# ssa = SSA(4, [2, 2], CONFIG.device)

# CONFIG.spectral_threshold = 1440
# CONFIG.dbn_hidden_layer_sizes = [80, 80, 80]

# gamma = 1
# reg_coeff = 1

# CONFIG.alpha = 0.2#0.779

# mat_q_trend_full, mat_q_resid_full = preprocess_data(CONFIG.spectral_threshold, mat_q)
# (_, val_full_trend_wd_dataloader, _), (_, val_full_trend_we_dataloader, _) = \
#   crop_and_split_mat(mat_q_trend_full, CONFIG, separate_weekends=False)
# (_, val_full_resid_wd_dataloader, _), (_, val_full_resid_we_dataloader, _) = \
#   crop_and_split_mat(mat_q_resid_full, CONFIG, separate_weekends=False)
# (_, val_full_overall_wd_dataloader, _), (_, val_full_overall_we_dataloader, _) = \
#   crop_and_split_mat(mat_q, CONFIG, separate_weekends=False)
# mat_q_we_full = torch.row_stack([d[1] for d in val_full_overall_we_dataloader])
# mat_q_trend_we_full = torch.row_stack([d[1] for d in val_full_trend_we_dataloader])
# mat_q_resid_we_full = torch.row_stack([d[1] for d in val_full_resid_we_dataloader])

# results_P = {}
# for alpha in range(...): # TBD
#   mat_c_repr, mat_x, nonempty, representatives = compress_data(
#       mat_q.abs(), CONFIG.read_period, CONFIG.train_period, CONFIG.alpha)
#   mat_q_trend_repr, mat_q_resid_repr = preprocess_data(
#       CONFIG.spectral_threshold, mat_c_repr)
#   mat_q_resid_repr = crop_q_between(mat_q_resid_repr, CONFIG.read_period,
#                                     CONFIG.train_period).real
#   mat_c_repr = crop_q_between(
#       mat_c_repr, CONFIG.read_period, CONFIG.train_period)

#   print('Representatives: ' + str(representatives.tolist()))
#   mat_c_trend_pred = []
#   mat_c_resid_pred = []
#   mat_c_pred = []
#   for section_index, section_number in enumerate(representatives.tolist()):#[[0, representatives.tolist()[2]]]
#     mat_q_trend = mat_q_trend_repr[:, section_index][:, None].real
#     mat_q_resid = mat_q_resid_repr[:, section_index][:, None].real

#     norm_const = normalizing_constants_all[section_number].item()
#     norm_const = 1

#     (_, val_trend_wd_dataloader, _), (_, val_trend_we_dataloader, _) = \
#       crop_and_split_mat(mat_q_trend, CONFIG, separate_weekends=False)
#     mat_c_wd_datasets, mat_c_we_datasets = split_mat(mat_q_resid, CONFIG)

#     del mat_q_resid

#     dbn, kelm, val_resid_we_dataloader = train_with_config(
#         CONFIG, mat_c_we_datasets, dbn_training_epochs=1, stride=1,
#         gamma=gamma, reg_coeff=reg_coeff)
#     prefix = f'alpha{CONFIG.alpha}_p{P}_n{N}_sec{section_number}_'\
#       .replace('.', '_')
#     CONFIG.save(dbn.state_dict(), prefix + 'dbn.pt')
#     CONFIG.save(kelm.state_dict(), prefix + 'kelm.pt')

#     iter_trend = iter(val_trend_we_dataloader)
#     n_samples = 0

#     trend = []
#     resid = []
#     overall = []

#     for X_resid, y_resid in val_resid_we_dataloader:
#         X_trend, y_trend = next(iter_trend)
#         pred_trend = ssa.forecast(
#             X_trend.squeeze(0).T, 1).sum(0)[-1][None]
#         trend.append(pred_trend.item())

#         pred_resid = dbn(X_resid).squeeze(0)
#         pred_resid = kelm(pred_resid).T
#         resid.append(pred_resid.item())

#         pred = pred_trend + pred_resid
#         overall.append(pred.item())

#         y = y_trend + y_resid

#         n_samples += 1

#     mat_c_trend_pred.append(trend)
#     mat_c_resid_pred.append(resid)
#     mat_c_pred.append(overall)

#     results_P[section_number] = {
#         'resid': resid,
#         'trend': trend,
#         'overall': overall
#     }

#     del dbn
#     del kelm
  
#   mat_c_trend_pred = torch.tensor(mat_c_trend_pred)
#   mat_c_resid_pred = torch.tensor(mat_c_resid_pred)
#   mat_c_pred = torch.tensor(mat_c_pred)

#   mat_c_trend_gt = mat_q_trend_we_full[:, representatives]
#   mat_c_resid_gt = mat_q_resid_we_full[:, representatives]
#   mat_c_gt = mat_q_we_full[:, representatives]

#   mat_x_trend = torch.linalg.pinv(mat_c_trend_gt) @ mat_q_trend_we_full
#   mat_x_resid = torch.linalg.pinv(mat_c_resid_gt) @ mat_q_resid_we_full

#   mat_q_we_pred = mat_c_pred @ mat_x
#   mat_q_trend_pred = mat_c_trend_pred @ mat_x_trend
#   mat_q_resid_pred = mat_c_resid_pred @ mat_x_resid

  

# results_we[P] = results_P
# print(results_we)

In [7]:
mat_c_trend_pred.shape

NameError: name 'mat_c_trend_pred' is not defined